# Training the electric network

Write a two-layer neural network to train image in raw format. The network is trained to predict the electric network from the image. The network is trained on the training set and evaluated on the test set. The network is trained using stochastic gradient descent with a fixed learning rate and no regularization.

In [1]:
# import the necessary packages
import torch as torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# import raw image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import os

## 1. have a feel of the raw image

In [2]:
def rawimage2tensor(raw_data, width=32, height=32):
    # create a PIL image from the raw data
    img = Image.frombytes('I;16', (width, height), raw_data, decoder_name='raw')

    # convert the PIL image to a numpy array
    image = np.array(img)

    # convert uint16 to int16
    image = image.astype(np.int16)

    # flatten image to 1D array
    image = image.flatten()

    # convert the numpy array to a tensor
    tensor = torch.from_numpy(image)

    # return the tensor
    return tensor

### 1.1 read the image

In [3]:
# # open the raw image file
# with open('test_image/pic_20231025_15 1 2_991.raw', 'rb') as f:
#     # read the raw image data
#     raw_data = f.read()

# initialize the dataset
dataset = []

# open the raw image file in folder '红外光测试'
for image in os.listdir('红外光测试/img32x32_20231023_113512_flight/'):
    with open('红外光测试/img32x32_20231023_113512_flight/' + image, 'rb') as f:
        # read the raw image data
        raw_data = f.read()
        matrix = rawimage2tensor(raw_data)
        label = 0  # 0 for flight
        # append the image tensor into dataset
        dataset.append((matrix, label))

for image in os.listdir('红外光测试/img32x32_20231023_113737_oil/'):
    with open('红外光测试/img32x32_20231023_113737_oil/' + image, 'rb') as f:
        # read the raw image data
        raw_data = f.read()
        matrix = rawimage2tensor(raw_data)
        label = 1  # 1 for oil-tank
        # append the image tensor into dataset
        dataset.append((matrix, label))

for image in os.listdir('红外光测试/img32x32_20231023_113921_boat/'):
    with open('红外光测试/img32x32_20231023_113921_boat/' + image, 'rb') as f:
        # read the raw image data
        raw_data = f.read()
        matrix = rawimage2tensor(raw_data)
        label = 2  # 2 for boat
        # append the image tensor into dataset
        dataset.append((matrix, label))

In [4]:
print(len(dataset))
print(dataset[0][0].shape)
# check a random sample in dataset
sample = dataset[0]
print(sample[0], sample[1])
# plt.imshow(sample[0], cmap='gray')

# sample = dataset[250]
# print(sample[0], sample[1])
# plt.imshow(sample[0], cmap='gray')

# sample = dataset[500]
# print(sample[0], sample[1])
# plt.imshow(sample[0], cmap='gray')

# import dataset into dataloader
train_dataloader = DataLoader(dataset, batch_size=1)

for batch, (X, y) in enumerate(train_dataloader):
    print(batch, X.shape, y.shape)
    break

750
torch.Size([1024])
tensor([4268, 4277, 4430,  ..., 4204, 2941, 2878], dtype=torch.int16) 0
0 torch.Size([1, 1024]) torch.Size([1])


## 2. make a dataset
3 kinds of objects in 3 folders, 250 raw images in each folder, labeled as 0, 1, 2.

`datasets.ImageFolder(root='./data', transform=transform)` do not support raw image, so we need to make a dataset by ourselves.

## 3. define the network

The input tensor is a 32 * 32 raw image and flatten to 1 * 1024.

The weight value in the 1st layer should be binary (+1 or -1), the dimension of the weight matrix is 1024*32. The activation function is relu function.

The weight value in the 2nd layer signed 16 bit, the dimension of the weight matrix is 32 *3. The output is 1 *3 matrix. The activation function is softmax function.

In [6]:
# define the network
# The weight value in the 1st layer should be binary (+1 or -1), 
# the dimension of the weight matrix is 1024*32. The activation function is relu function.
# The weight value in the 2nd layer signed 16 bit, the dimension of the weight matrix is 32 *3. 
# The output is 1 *3 matrix. The activation function is softmax function.

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(32*32, 32)
        self.fc2 = nn.Linear(32, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## 4. train the network

In [7]:
model = Network()
print(model)

learning_rate = 1e-3
batch_size = 25
epochs = 5

loss_fn = nn.CrossEntropyLoss()
print(loss_fn)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(optimizer)

Network(
  (fc1): Linear(in_features=1024, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=3, bias=True)
)
CrossEntropyLoss()
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [9]:

size = len(train_dataloader.dataset)
# Set the model to training mode - important for batch normalization and dropout layers
# Unnecessary in this situation but added for best practices
model.train()
for batch, (X, y) in enumerate(train_dataloader):
    # Compute prediction and loss
    X = X.to(torch.float32)
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


loss: 2531.258789  [    1/  750]
loss: 0.000000  [  101/  750]
loss: 0.000000  [  201/  750]
loss: 0.914605  [  301/  750]
loss: 0.862225  [  401/  750]
loss: 1.328979  [  501/  750]
loss: 0.000000  [  601/  750]
loss: 0.000000  [  701/  750]
